In [2]:
import spacy

from spacy.lang.en import English

nlp = English()

tokens = nlp("Some\nspaces  and\ttab characters")
tokens_text = [t.text for t in tokens]

assert tokens_text == ["Some", "\n", "spaces", " ", "and", "\t", "tab", "characters"]

In [1]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

In [4]:
doc = nlp(["This is a sentence. This is another.", 
           "Today is a good day."])

list(doc.sents)

TypeError: Argument 'string' has incorrect type (expected str, got list)